In [294]:
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta as tmd

from warnings import filterwarnings
filterwarnings('ignore')

from importlib import reload
import xAPIConnector.xAPIConnector
reload(xAPIConnector.xAPIConnector)
from xAPIConnector.xAPIConnector import *


import xAPIConnector.DataLoader
reload(xAPIConnector.DataLoader)
from xAPIConnector.DataLoader import *

import xAPIConnector.config
reload(xAPIConnector.config)
from xAPIConnector.config import user_id, pwd

In [296]:
symbols = ['BITCOIN', 'ETHEREUM']
start = "2025-01-08 00:00:00"

In [300]:
dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start)
data

[2025-01-13 14:10:23,236][connect:59] Socket connected


[2025-01-13 14:10:23.236707] Loguję do API...
	Wysyłam zapytanie do API... {'status': True, 'returnData': {'rateInfos': [{'ctm': 1736204460000, 'ctmString': 'Jan 7, 2025, 12:01:00 AM', 'open': 1019550.0, 'close': -880.0, 'high': 0.0, 'low': -930.0, 'vol': 204.0}, {'ctm': 1736204520000, 'ctmString': 'Jan 7, 2025, 12:02:00 AM', 'open': 1018650.0, 'close': -500.0, 'high': 0.0, 'low': -530.0, 'vol': 157.0}, {'ctm': 1736204580000, 'ctmString': 'Jan 7, 2025, 12:03:00 AM', 'open': 1018220.0, 'close': -80.0, 'high': 410.0, 'low': -320.0, 'vol': 188.0}, {'ctm': 1736204640000, 'ctmString': 'Jan 7, 2025, 12:04:00 AM', 'open': 1018110.0, 'close': 240.0, 'high': 420.0, 'low': -130.0, 'vol': 176.0}, {'ctm': 1736204700000, 'ctmString': 'Jan 7, 2025, 12:05:00 AM', 'open': 1018180.0, 'close': -310.0, 'high': 0.0, 'low': -440.0, 'vol': 181.0}, {'ctm': 1736204760000, 'ctmString': 'Jan 7, 2025, 12:06:00 AM', 'open': 1017940.0, 'close': 1050.0, 'high': 1080.0, 'low': -50.0, 'vol': 179.0}, {'ctm': 173620482

,BITCOIN,ETHEREUM
Date,,
2025-01-07 00:01:00,101867.0,3671.000
2025-01-07 00:02:00,101815.0,3667.995
2025-01-07 00:03:00,101814.0,3666.930
2025-01-07 00:04:00,101835.0,3668.180
2025-01-07 00:05:00,101787.0,3664.715
...,...,...
2025-01-13 14:05:00,90813.2,3049.630
2025-01-13 14:06:00,90807.9,3051.240
2025-01-13 14:07:00,90859.8,3052.255


# Klasyfikacja

Klasyfikatory można budować na mnóstwo różnych sposobów. Ogólnie trzeba się zdecydować, na ilu obserwacjach wstecz ma się opierać klasyfikacja. Liczbę tych obserwacji nazwiemy 'window' i przyjmiemy jako parametr.

Na razie ustawiamy klasyfikację na podstawie poprzednich 4 godzin.

Pierwszy pomysł to klasyfikacja na podstawie stóp zwrotu, klasy również będziemy budować na podstawie stóp zwrotu. Metoda budowania klas zostanie omówiona dalej.

In [188]:
interval = '5m' # dane 5-minutowe
returns = prices.pct_change().dropna()

window = int(4*60/int(interval[:-1]))

In [198]:
def generate_X_y(data: pd.DataFrame | pd.Series, window: int, skip: int) -> tuple:   
    # można ustawić okna zachodzące (0 < skip < window)
    # można ustawić niezachodzące (skip >= window)
    assert skip > 0, "Pętla w kodzie nigdy się nie zakończy..."
    
    # Generujemy 'okna'
    X = pd.DataFrame(columns=range(window-1))
    y = pd.Series()
    
    i = len(data)
    count = 0
    while i >= window:
        temp_y = data.iloc[i-window:i]
        
        X.loc[count, :] = temp_y.iloc[:-1].values
        y.loc[count] = temp_y.iloc[-1]
        
        i = i - skip    
        count += 1
        
    return np.array(X), np.array(y)

### Regresja logistyczna

In [237]:
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import LeaveOneOut

Pierwszy pomysł na zdefiniowanie klas to przyjęcie '1' jeśli stopa zwrotu jest $>threshold$ oraz '0' w p.p.

Próg odcięcia można podać jako parametr i już po pierwszych testach widać, że dla prostej klasyfikacji typu "czy zwrot > 0" skuteczność jest około $50\%$. Im wyższy próg, tym lepsza skuteczność.

In [244]:
skip = window
X, y = generate_X_y(data=returns, window=window, skip=skip)

threshold = 0.002
y = (np.array(y) > threshold)*1.0

clf = LogisticRegression().fit(X, y)
print(f"Skuteczność treningowa modelu: {clf.score(X, y):.4%}")

Skuteczność treningowa modelu: 87.0968%


Teraz podzielimy dane na treningowe i testowe, po czym sprawdzimy jakość predykcji

In [245]:
train_test_ratio = 0.7
len_train = int(train_test_ratio*X.shape[0])

Xtrain = X[:len_train, :]
ytrain = y[:len_train]

Xtest = X[len_train:, :]
ytest = y[len_train:]

print(f"[INFO] W zbiorze treningowym znaduje się {Xtrain.shape[0]} obserwacji.")
print(f"[INFO] W zbiorze testowym znajduje się {Xtest.shape[0]} obserwacji.")

[INFO] W zbiorze treningowym znaduje się 21 obserwacji.
[INFO] W zbiorze testowym znajduje się 10 obserwacji.


In [246]:
clf = LogisticRegression().fit(Xtrain, ytrain)
print(f"Skuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"Skuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

Skuteczność treningowa modelu: 90.4762%
Skuteczność testowa modelu: 80.0000%
